# MNIST Digit Classification Model using CNN
Reference: https://github.com/boaaaang/CNN-Implementation-in-Verilog

## 1. Model Development
Create and train the CNN model for MNIST digit classification.

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
from matplotlib import pyplot

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

100%|██████████| 9.91M/9.91M [00:00<00:00, 40.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.10MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 10.1MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.7MB/s]


In [ ]:
# Data Loader (Input Pipeline)
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
# CNN class
class CNN(nn.Module):
     # Initialization
    def __init__(self):
        super (CNN, self).__init__()

        self.conv1_out_np = np.zeros((1, 3, 24, 24))
        self.mp1_out_np = np.zeros((1, 3, 11, 11))
        #self.conv2_out_np = np.zeros((1, 3, 8, 8))
       # self.mp2_out_np = np.zeros((1, 3, 4, 4))
        self.fc_in_np = np.zeros((1,363))
        self.fc_out_np = np.zeros((1, 10))

        # 1st Convolution Layer
        # Image Input Shape -> (28, 28, 1)
        # Convolution Layer -> (22, 22, 3)
        # Pooling Max Layer -> (11, 11, 3)
        self.conv1 = nn.Conv2d(1, 3, kernel_size=7)

        # 2nd Convolution Layer
        # Image Input Shape -> (11, 11, 3)
        # Convolution Layer -> (8, 8, 3)
        # pooling Max Layer -> (4, 4, 3)
       # self.conv2 = nn.Conv2d(3, 3, kernel_size=4)

        # Max Pooling Layer
        self.mp = nn.MaxPool2d(2)

        # Fully Connected Layer
        # Num of Weight = 480
        self.fc_1 = nn.Linear(363, 10)

    def forward(self, x):
        in_size = x.size(0)

        # Layer Integration
        x = self.conv1(x)
        self.conv1_out_np = x.detach().numpy()

        x = F.relu(self.mp(x))
        self.mp1_out_np = x.detach().numpy()

       # x = self.conv2(x)
        #self.conv2_out_np = x.detach().numpy()

       # x = F.relu(self.mp(x))
        #self.mp2_out_np = x.detach().numpy()

        # Flatten Layer
        x = x.view(in_size, -1)
        self.fc_in_np = x.detach().numpy()

        # Fully Connected Layer
        x = self.fc_1(x)
        self.fc_out_np = x.detach().numpy()

        return F.log_softmax(x)

In [ ]:
# Instantiation
model = CNN()
print(model)

CNN(
  (conv1): Conv2d(1, 3, kernel_size=(7, 7), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_1): Linear(in_features=363, out_features=10, bias=True)
)


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")

Number of parameters: 3790


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
# Training
def train(epoch):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()

        # Ouput of feedforwarding
        output = model(data)

        # Loss calibration
        loss = F.nll_loss(output, target)

        # Gradient
        loss.backward()

        # Back propagation
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
# Test
def test():
    model.eval()

    test_loss = 0
    correct = 0

    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)

        # Output of feedforwarding
        output = model(data)

        test_loss += F.nll_loss(output, target, size_average=False).item()

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
# Traning process
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300496
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.297985
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.272013
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.212318
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.166629
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.034612
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.819755
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.620409
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.324277
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.018335
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.909195
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.046364
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.789468
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.579589
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.856346
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.798937
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.634326
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.522350
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.436259
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.518363
Train Epoch: 1 [12800/60000 (

## 2. Save Trained Weight and Bias
Save trained weight and bias to a .pt file.

In [ ]:
# Save model
torch.save(model, "./cnn_mnist.pt")

In [ ]:
# Load model
model = torch.load("./cnn_mnist.pt", weights_only=False)
model.eval()
print(model)

CNN(
  (conv1): Conv2d(1, 3, kernel_size=(7, 7), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_1): Linear(in_features=363, out_features=10, bias=True)
)


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")

Number of parameters: 3790


## 3. Extract Weight and Bias
Extract weight and bias to .mem files for RTL development.

In [ ]:
#################### Weight & Bias in HEX of Convolution Layer1 ####################

# Calibration
int_conv1_weight_1 = torch.tensor((model.conv1.weight.data[0][0] * 128), dtype = torch.int32)
int_conv1_weight_2 = torch.tensor((model.conv1.weight.data[1][0] * 128), dtype = torch.int32)
int_conv1_weight_3 = torch.tensor((model.conv1.weight.data[2][0] * 128), dtype = torch.int32)
int_conv1_bias = torch.tensor((model.conv1.bias.data * 128), dtype = torch.int32)

print("Signed")
print(int_conv1_weight_1)
print(int_conv1_weight_2)
print(int_conv1_weight_3)
print(int_conv1_bias)

# 2's Complement
for i in range(7):
    for j in range(7):
        if int_conv1_weight_1[i][j] < 0:
            int_conv1_weight_1[i][j] += 256
        if int_conv1_weight_2[i][j] < 0:
            int_conv1_weight_2[i][j] += 256
        if int_conv1_weight_3[i][j] < 0:
            int_conv1_weight_3[i][j] += 256

for k in range(3):
    if int_conv1_bias[k] < 0:
        int_conv1_bias[k] += 256

print ("Unsigned")
print(int_conv1_weight_1)
print(int_conv1_weight_2)
print(int_conv1_weight_3)
print(int_conv1_bias)

np.savetxt('conv1_weight_1.mem', int_conv1_weight_1, fmt='%1.2x',delimiter = " ")
np.savetxt('conv1_weight_2.mem', int_conv1_weight_2, fmt='%1.2x',delimiter = " ")
np.savetxt('conv1_weight_3.mem', int_conv1_weight_3, fmt='%1.2x',delimiter = " ")
np.savetxt('conv1_bias.mem', int_conv1_bias, fmt='%1.2x',delimiter = " ")

Signed
tensor([[  3,   0,  -4,  17, -12, -18,  21],
        [ 21,  16,  30,  32,   3,   1,   1],
        [ 11,  19,  61,  44,   1, -30, -22],
        [-18,  15,  74,  74, -10, -45, -59],
        [-18,  22,  85,  82,   9, -16, -52],
        [ -5, -13,  60,  70,  82,  36,  -2],
        [-22, -31,  15,  51,  47,  74,  44]], dtype=torch.int32)
tensor([[-24, -36,  -8, -20,   0,  31,  42],
        [-36,   4,  -1,  50,  65,  53,  70],
        [-11,   5,  65,  77,  79,  60,  22],
        [ 11,  19,  45,  56,  35,   6, -41],
        [ 15,  13,  56,  36,  -3, -41, -58],
        [  5,   9,  25, -12, -46, -61, -62],
        [  2,  -2,  -8, -47, -59, -52, -51]], dtype=torch.int32)
tensor([[  11,   48,   61,   66,   90,   78,   63],
        [  -9,   22,   46,   62,   54,   63,   66],
        [ -49,  -28,  -23,  -32,  -21,  -24,   16],
        [ -68,  -79, -102, -104,  -95,  -74,  -39],
        [ -13,  -70,  -90,  -90,  -82,  -68,  -45],
        [   6,    4,   -7,    5,   -9,   -1,  -20],
        [  

In [ ]:
#################### Weight & Bias in HEX of Fully Connected Layer ####################

print(np.shape(model.fc_1.weight))
print((model.fc_1.weight * 128).int())

print(np.shape(model.fc_1.bias))
print((model.fc_1.bias * 128).int())

int_fc_weight = (model.fc_1.weight * 128).int()
int_fc_bias = (model.fc_1.bias * 128).int()

# 2's Complement
for i in range(10):
    for j in range(363):
        if int_fc_weight[i][j] < 0 :
            int_fc_weight[i][j] += 256
    if int_fc_bias[i] < 0 :
        int_fc_bias[i] += 256

print(int_fc_weight)
print(int_fc_bias)

np.savetxt('fc_weight.mem', int_fc_weight, fmt='%1.2x',delimiter = " ")
np.savetxt('fc_bias.mem', int_fc_bias, fmt='%1.2x',delimiter = " ")

torch.Size([10, 363])
tensor([[-4, -3, -2,  ...,  3, -3, -1],
        [ 2, -2,  0,  ..., 12,  8,  3],
        [-6, -4, -4,  ..., 17, 16,  2],
        ...,
        [ 1,  2,  8,  ..., -7,  2,  4],
        [ 1, -8, -4,  ..., -9, -1,  1],
        [ 1,  3, -5,  ..., -6, -1, -5]], dtype=torch.int32)
torch.Size([10])
tensor([  3,  14,  -8, -19,   6,  14,  -2,  11, -15,  -8], dtype=torch.int32)
tensor([[252, 253, 254,  ...,   3, 253, 255],
        [  2, 254,   0,  ...,  12,   8,   3],
        [250, 252, 252,  ...,  17,  16,   2],
        ...,
        [  1,   2,   8,  ..., 249,   2,   4],
        [  1, 248, 252,  ..., 247, 255,   1],
        [  1,   3, 251,  ..., 250, 255, 251]], dtype=torch.int32)
tensor([  3,  14, 248, 237,   6,  14, 254,  11, 241, 248], dtype=torch.int32)
